In [105]:
import os
import numpy as np
import cv2
from collections import Counter

output_dir = 'assets/lab7_output'

Сохранение монохромного изображения в виде текстового или бинарного
файла (в папку assets/lab7_output).

In [106]:
img_path = 'assets/lab7.jpg'
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

txt_path = os.path.join(output_dir, 'image.txt')
np.savetxt(txt_path, img, fmt='%d')

Реализуем алгоритм вейвлет-преобразования Хаара для изображения.

In [107]:
def haar(image):
    rows, cols = image.shape
    image_float = image.astype(np.float64)
    row_transform = np.zeros_like(image_float)

    for i in range(rows):
        for j in range(0, cols - 1, 2):
            row_transform[i, j // 2] = (image_float[i, j] + image_float[i, j + 1]) / 2
            row_transform[i, (j // 2) + cols // 2] = (image_float[i, j] - image_float[i, j + 1]) / 2

    result = np.zeros_like(row_transform)
    for j in range(cols):
        for i in range(0, rows - 1, 2):
            result[i // 2, j] = (row_transform[i, j] + row_transform[i + 1, j]) / 2
            result[(i // 2) + rows // 2, j] = (row_transform[i, j] - row_transform[i + 1, j]) / 2

    LL = result[:rows // 2, :cols // 2]
    LH = result[:rows // 2, cols // 2:]
    HL = result[rows // 2:, :cols // 2]
    HH = result[rows // 2:, cols // 2:]
    return LL, LH, HL, HH

LL, LH, HL, HH = haar(img)

print(f"LL: {LL.shape}, LH: {LH.shape}, HL: {HL.shape}, HH: {HH.shape}")


LL: (289, 289), LH: (289, 290), HL: (290, 289), HH: (290, 290)


Выполним квантование высокочастотных компонент.

In [108]:
q = 4

def quant(coeffs, q):
    min_val = np.min(coeffs)
    max_val = np.max(coeffs)
    step = (max_val - min_val) / q
    quantized = np.round((coeffs - min_val) / step).astype(int)
    return quantized

LH_q = quant(LH, q)
HL_q = quant(HL, q)
HH_q = quant(HH, q)

Сохраним получившийся массив значений в текстовый или бинарный файл в порядке LL,
LH, HL, HH вейвлет-преобразования Хафа.

In [109]:
os.makedirs(output_dir, exist_ok=True)

def run_length_encode(data):
    encoded = []
    for value, count in Counter(data.flatten()).items():
        encoded.append((value, count))
    return encoded

LH_rle = run_length_encode(LH_q)
HL_rle = run_length_encode(HL_q)
HH_rle = run_length_encode(HH_q)

output_file = os.path.join(output_dir, 'haar_image.txt')

with open(output_file, 'w') as f:
    np.savetxt(f, LL, fmt='%d')

    f.write("LH:\n")
    for value, count in LH_rle:
        f.write(f"{value} {count}\n")

    f.write("HL:\n")
    for value, count in HL_rle:
        f.write(f"{value} {count}\n")

    f.write("HH:\n")
    for value, count in HH_rle:
        f.write(f"{value} {count}\n")

Сравним объем памяти, занимаемый исходным изображением, и
изображением, полученным после преобразования Хафа и сжатием длин серий.

In [110]:
haar_storage_size = os.path.getsize(output_file)

original_txt = os.path.join(output_dir, "image.txt")
original_size = os.path.getsize(original_txt)

print(f"Исходное озбражение: {original_size} байт")
print(f"После сжатия: {haar_storage_size} байт")
print(f"Коэффициент сжатия: {original_size / haar_storage_size:.2f}")

Исходное озбражение: 1316232 байт
После сжатия: 328185 байт
Коэффициент сжатия: 4.01
